In [380]:
from mip import (CBC, CONTINUOUS, MAXIMIZE, Model, xsum, OptimizationStatus)
import numpy as np

In [396]:
def readFile(filename):
    File = open(filename, "r")
    lines = File.readlines()
    File.close()
    
    # separa os arrays em '\n'
    lines = [lines.split() for lines in lines]

    # número de variáveis
    # número de restrições
    num_vars, num_res = int(lines[0][0]), int(lines[0][1])

    lines = lines[1:]

    # coeficientes da função objetivo
    coef_fo = lines[0]
    coef_fo = [float(i) for i in coef_fo]

    # coeficientes das restrições
    coef_restantes = lines[1:]
    coef_restantes = [[float(i) for i in coef_restantes[j]] for j in range(num_res)]

    coef_left = [coef_restantes[j][:-1] for j in range(num_res)]

    coef_right = [coef_restantes[j][-1] for j in range(num_res)]


    m = Model(sense=MAXIMIZE, solver_name=CBC)

    # variáveis de decisão
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name="x_"+str(i)) for i in range(num_vars)]

    # função objetivo
    m.objective = xsum(coef_fo[i] * x[i] for i in range(num_vars))

      #cria as restrições
      #model += xsum(coefientes[i]*x[i] for i in range(num_vars)) <= coefientes[-1] 

    # restrições
    for i in range(num_res):
        m += xsum(coef_left[i][j] * x[j] for j in range(num_vars)) <= coef_right[i]
    # 'silencia' o solver
    m.verbose = 0

    return m

In [397]:
m = readFile('example.txt')

Utilizaremos implementação em pilha para desenvolver o projeto.

In [398]:
class No:
    def __init__ (self, _modelo):
        _modelo.optimize()
        _modelo.verbose = 0
        self.modelo = _modelo.copy()
        self.Z = self.modelo.objective_value
        self.vars = self.modelo.vars


In [399]:
def poda_integralidade (modelo: Model):
    # Se todas as variáveis são inteiras, o nó é viável
    # logo, deve ser podado
    if all(np.isclose(var.x, np.round(var.x)) for var in modelo.vars):
        return True
    else:
        return False

# poda por limitante, analisa o valor da função objetivo
def poda_limitante (lim_inf: float, Z: int):
    # Se o valor da função objetivo for menor ou igual ao limitante inferior,
    # qualquer nó abaixo incluindo o próprio nó é descartado,
    # pois já há uma solução melhor encontrada
    if Z is not None and Z <= lim_inf:
        return True
    else:
        return False

# poda por inviabilidade, analisa o status do modelo
def poda_inviabilidade (modelo: Model):
    if not modelo.status == OptimizationStatus.OPTIMAL:
        return True
    # Inviável
    #if modelo.status == OptimizationStatus.INFEASIBLE:
    #    return True
    # Não encontrou solução
    #elif modelo.status == OptimizationStatus.NO_SOLUTION_FOUND:
    #    return True
    
    return False

In [400]:
def print_no(filho: Model):
    print("Status = ", filho.status)
    print(f"Solution value  = {filho.objective_value:.2f}\n")

    print("Solution:")
    for v in filho.vars:
        print(f"{v.name} = {v.x:.2f}")

In [401]:
def closest_value(array, value): 
    array = np.asarray(array)
    
    value_found = np.absolute(array - value)
    value_found = value_found.argmin()
    
    return value_found 

In [402]:
# Gera os dois nós filhos do algoritmo a partir da variável mais próxima de 0.5
def passar_no (pai):
    # escolhe a variável mais próxima de 0.5
    dif_abs = 1
    for var in pai.modelo.vars:
        if abs (0.5 - var.x) <= dif_abs:
            dif_abs = abs (0.5 - var.x)
            var_escolhida = var.name
    
    # cria os modelos filhos
    # e adiciona restrição para a variável escolhida
    m1 = pai.modelo.copy()
    m1.verbose = 0
    m1 += m1.vars[var_escolhida] == 0

    m2 = pai.modelo.copy()
    m2.verbose = 0
    m2 += m2.vars[var_escolhida] == 1

    # cria os nós filhos
    filho1 = No(m1)
    filho2 = No(m2)

    return filho1, filho2


In [403]:
class Branch_and_Bound:
    def __init__ (self, no_modelo):
        self.raiz = no_modelo

    def execute(self):
        lim_inf = float('-inf')
        fronteira = [self.raiz]
        solucao = "Nenhuma solução encontrada."
        
        while len(fronteira):
            no = fronteira[0]

            if poda_integralidade(no.modelo):
                if no.Z is not None and no.Z > lim_inf:
                    lim_inf = no.Z
                    solucao  = no.modelo
                fronteira.pop(0)
                continue

            if poda_inviabilidade(no.modelo):
                fronteira.pop(0)
                continue

            if poda_limitante(lim_inf, no.Z):
                fronteira.pop(0)
                continue

            filho1, filho2 = passar_no(no)
            fronteira.append(filho1)
            fronteira.append(filho2)
            del fronteira[0]

        print_no(solucao)

In [404]:
modelo1 = readFile('example.txt')
teste1 = No(modelo1)

In [405]:
BnB = Branch_and_Bound(teste1)

In [406]:
BnB.execute()

TypeError: type NoneType doesn't define __round__ method